In [2]:
import pandas as pd
import subprocess
from subprocess import Popen, PIPE
import sys
sys.path.append('/home/emilia/miniconda3/lib/python3.12/site-packages')
sys.path.append('/home/emilia/miniconda3/envs/my_rdkit_env/lib/python3.10/site-packages')
from rdkit import Chem
from rdkit.Chem import SDWriter

In [3]:
target = 'fa10_c2'
N_cores = 19
outputs_dir = '/media/emilia/HDD/Documents/Dockings/goldilocks/fa10/' + target +'/out_' + target + '_dock6.sdf'

## Joins the flex_out files

We need the flex_out_score_path all in one file, but if there are more than 1 core, then when have multiple files. The next code joins all of them.

In [13]:
import os

def join_mol2_files(output_file, num_cores):
    if not os.path.exists(output_file):
        open(output_file, 'w').close()  # Create an empty file if it doesn't exist
    
    with open(output_file, 'a') as outfile:
        for i in range(1, num_cores + 1):
            #input_file = f'/home/emilia/Documents/Dockings/goldilocks/' + target +  f'/{i}/flex.out_scored.mol2'
            input_file = f'/media/emilia/HDD/Documents/Dockings/goldilocks/fa10/' + target + '/'+ f'{i}/flex.out_scored.mol2'

            try:
                with open(input_file, 'r') as infile:
                    outfile.write(infile.read())
            except FileNotFoundError:
                print(f"File '{input_file}' not found.")

# Example usage:
flex_out_score_path = '/media/emilia/HDD/Documents/Dockings/goldilocks/fa10/' + target +'/flex.out_scoredFinal.mol2'
num_cores = N_cores  # Define the value of N_cores
join_mol2_files(flex_out_score_path, num_cores)


In [7]:
flex_out_score_path = '/media/emilia/HDD/Documents/Dockings/goldilocks/fa10/' + target +'/flex.out_scoredFinal.mol2'



## convertir .mol2 a .sdf

This new .sdf is missing the IX column, it'll be added in the next step with the score

In [5]:
molecule_names = []

ruta_carpeta = '/media/emilia/HDD/Documents/Dockings/goldilocks/fa10/' + target
ruta_docking = ruta_carpeta +"/"
for i in range (N_cores):
    ruta_dock =  ruta_docking  + str(i+1) + "/dock.out"
    with open(ruta_dock, 'r') as file:
        for line in file:
            if line.startswith("Molecule:"):
                molecule_info = line.strip()
                molecule_names.append(molecule_info.split()[1])
print(len(molecule_names))



416481


In [16]:
subprocess.Popen(['obabel','-imol2',flex_out_score_path,'-osdf','-O',outputs_dir])

<Popen: returncode: None args: ['obabel', '-imol2', '/media/emilia/HDD/Docum...>

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is ZINC000095988747 312523)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is ZINC000091434311 312538)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is ZINC000091468671 312542)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is ZINC000077422598 312644)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is ZINC000000090652 312709)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is ZINC000002640493 312764)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is ZINC000003224032 312789)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL2 file (title is ZINC000012874356 312847)



In [4]:
mols_sdf = 0
#312352
file_sdf = open (outputs_dir, "r")
lines=file_sdf.readlines()
for line in lines: 
    if line.startswith("$$$$"):
        mols_sdf = mols_sdf + 1
mols_sdf

416317

In [5]:
f=open(outputs_dir,'r')
lines=f.readlines()
ix=[i for i in range(len(lines)) if lines[i]=='$$$$\n']
ix.insert(0,-1)

Nmols=len(ix)-1
Nmols

416317

In [8]:
f_mol2=open(flex_out_score_path,'r')
lines_mol2=f_mol2.readlines()

scores= [l.split()[2] for l in lines_mol2 if l.startswith('##########                          Grid_Score:') ]


## Add column IX and Score

In [9]:
new_file=[]

for i in range(len(ix)-1):
    mol=lines[ix[i]+1:ix[i+1]-1]
    IX=mol[0].split(' ')[1].strip()
    mol.append('M  END\n')
    mol.append('>  <IX> \n')
    mol.append(str(IX)+' \n')
    mol.append('>  <Score_dock6> \n')
    mol.append(scores[i]+' \n')
    mol.append(' \n')
    mol.append('$$$$\n')
    
    for line in mol:
        new_file.append(line)


In [11]:
f=open(outputs_dir.split("_")[0]+"_"+target+"_dock6.sdf",'w')
for line in new_file:
    f.write(line)
f.close()

In [10]:
mols_new_sdf = 0
file_new_sdf = open (outputs_dir.split("_")[0]+"_"+target+"_dock6.sdf", "r")
lines=file_new_sdf.readlines()
for line in lines: 
    if line.startswith("$$$$"):
        mols_new_sdf = mols_new_sdf + 1
mols_new_sdf

FileNotFoundError: [Errno 2] No such file or directory: '/media/emilia/HDD/Documents/Dockings/goldilocks/fa10/fa10_fa10_c1_dock6.sdf'